## Imports

In [1]:
import pandas as pd
from lcanalyzer import models

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


## Params

In [2]:
lc_datasets = {}
# Create an empty dict
lc = {}
# Define the bands names
bands = 'ugrizy'

colname_mag = 'psfMag'

## Data loading

In [3]:
lc_datasets['lsst'] = pd.read_pickle('data/lsst_RRLyr.pkl')
lc_datasets['lsst'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11177 entries, 0 to 11176
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   band         11177 non-null  object 
 1   ccdVisitId   11177 non-null  int64  
 2   coord_ra     11177 non-null  float64
 3   coord_dec    11177 non-null  float64
 4   objectId     11177 non-null  int64  
 5   psfFlux      11177 non-null  float64
 6   psfFluxErr   11177 non-null  float64
 7   psfMag       10944 non-null  float64
 8   ccdVisitId2  11177 non-null  int64  
 9   band2        11177 non-null  object 
 10  expMidptMJD  11177 non-null  float64
 11  zeroPoint    11177 non-null  float32
dtypes: float32(1), float64(6), int64(3), object(2)
memory usage: 1004.3+ KB


## Data inspection

In [4]:
lc_datasets['lsst'].head()

,band,ccdVisitId,coord_ra,coord_dec,objectId,psfFlux,psfFluxErr,psfMag,ccdVisitId2,band2,expMidptMJD,zeroPoint
0,y,1032263018,62.462569,-44.11336,1251384969897480052,-515.183603,1697.218490,NaN,1032263018,y,61100.069706,30.602301
1,y,1033987172,62.462569,-44.11336,1251384969897480052,3151.738459,1686.955775,22.653625,1033987172,y,61102.068464,30.606100
2,u,675163080,62.462569,-44.11336,1251384969897480052,183.449123,209.242045,25.741211,675163080,u,60582.247144,30.469101
3,y,443055067,62.462569,-44.11336,1251384969897480052,-704.848327,1624.400086,NaN,443055067,y,60215.203585,30.612801
4,u,466722002,62.462569,-44.11336,1251384969897480052,382.472233,278.926670,24.943500,466722002,u,60261.078221,30.461201


## Selecting light curves for a single object

In [5]:
### Pick an object
obj_id = lc_datasets['lsst']['objectId'].unique()[4]

In [6]:
### Get all the observations for this obj_id for each band
# For each band create a bool array that indicates
# that this observation belongs to a certain object and is made in a
# certain band
for b in bands:
    filt_band_obj = (lc_datasets['lsst']['objectId'] == obj_id) & (
        lc_datasets['lsst']['band'] == b
    )
    # Select the observations and store in the dict 'lc'
    lc[b] = lc_datasets['lsst'][filt_band_obj]

## Trying the model.py functions

In [7]:
models.max_mag(lc['g'], colname_mag)

19.183367224358136

## Test development

In [8]:
test_input = pd.DataFrame(data=[[1, 5, 3], [7, 8, 9], [3, 4, 1]], columns=list("abc"))
test_output = 7
models.max_mag(test_input, "a") == test_output

True

In [9]:
models.calc_stat(lc, bands, colname_mag, stat='max')

{'u_max': 21.14104463470903,
 'g_max': 19.183367224358136,
 'r_max': 18.88863489356158,
 'i_max': 18.827676664867155,
 'z_max': 18.869036901698408,
 'y_max': 18.854156410782494}

In [20]:
stat = 'min'
df1 = pd.DataFrame(data=[[1, 5, 3], [7, 8, 9], [3, 4, 1]], columns=list("abc"))
df2 = pd.DataFrame(data=[[7, 3, 2], [8, 4, 2], [5, 6, 4]], columns=list("abc"))
df3 = pd.DataFrame(data=[[2, 6, 3], [1, 3, 6], [8, 9, 1]], columns=list("abc"))
test_input = {"df1": df1, "df2": df2, "df3": df3}
test_output = {"df1_"+stat: 4, "df2_"+stat: 3, "df3_"+stat: 3}
test_output == models.calc_stat(test_input, ["df1", "df2", "df3"], "b", stat=stat)

True

In [21]:
test_input

{'df1':    a  b  c
 0  1  5  3
 1  7  8  9
 2  3  4  1,
 'df2':    a  b  c
 0  7  3  2
 1  8  4  2
 2  5  6  4,
 'df3':    a  b  c
 0  2  6  3
 1  1  3  6
 2  8  9  1}

In [26]:
df1 = pd.DataFrame(data=[[1, 5, 3], [7, 8, 9], [3, 4, 1]], columns=list("abc"))
df2 = pd.DataFrame(data=[[7, 3, 2], [8, 4, 2], [5, 6, 4]], columns=list("abc"))
df3 = pd.DataFrame(data=[[2, 6, 3], [1, 3, 6], [8, 9, 1]], columns=list("abc"))
test_input = {"df1": df1, "df2": df2, "df3": df3}
test_output = {"df1_mean": 17/3., "df2_mean": 13/3., "df3_mean": 9}
models.calc_stat(test_input, ["df1", "df2", "df3"], "b", stat='mean') == test_output

False

In [27]:
test_output

{'df1_mean': 5.666666666666667, 'df2_mean': 4.333333333333333, 'df3_mean': 9}

In [28]:
models.calc_stat(test_input, ["df1", "df2", "df3"], "b", stat='mean')

{'df1_mean': 5.666666666666667, 'df2_mean': 4.333333333333333, 'df3_mean': 6.0}